# Transformer

## Config
### Imports

In [1]:
import os
import math
import inspect
from dataclasses import dataclass
import tiktoken

import torch
import torch.nn as nn
from torch.nn import functional as F

## Code

In [2]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
class Tokenizer():
    """A tokenizer class for encoding/decoding text sequences."""

    def __init__(self):
        """
        Constructor method to initialize special token indices and tokenizer encoding. 
        """
        # Initialize special token indices
        self.BOS_IDX: int = 100264  # Index for the Beginning of Sentence token
        self.EOS_IDX: int = 100265  # Index for the End of Sentence token
        self.PAD_IDX: int = 100266  # Index for the Padding token

        # Initialize base encoding from tiktoken
        cl100k_base = tiktoken.get_encoding("cl100k_base")

        # Initialize the tokenizer's encoding with special tokens added
        self.encoder = tiktoken.Encoding(
            name="cl100k_bep", # Name for the encoder with BOS, EOS, and PAD tokens added
            pat_str=cl100k_base._pat_str, # Pattern string from the base encoding
            mergeable_ranks=cl100k_base._mergeable_ranks, # Mergeable ranks from the base encoding
            special_tokens={
                **cl100k_base._special_tokens, # Special tokens from the base encoding
                "<|bos|>": self.BOS_IDX,  # BOS token
                "<|eos|>": self.EOS_IDX,  # EOS token
                "<|pad|>": self.PAD_IDX,  # PAD token
            }
        )
        
    def vocab_size(self) -> int:
        """
        Method to return the size of the vocabulary in the tokenizer's encoding.

        Returns:
            int: The size of the vocabulary.
        """
        return self.encoder.n_vocab
        
    def sequence_padding(self, sequence, max_size: int = 512) -> torch.Tensor:
        """
        Method to add BOS/PAD/EOS special tokens and ensure the sequence length is within the maximum size.

        Args:
            sequence (torch.Tensor or list): The input sequence.
            max_size (int, optional): The maximum allowed size for the sequence. Defaults to 512.

        Returns:
            torch.Tensor: The processed sequence with special tokens added and length limited.
        """
        # Ensure the sequence is a torch tensor
        tensor_sequence = torch.tensor(sequence, dtype=torch.long) if not torch.is_tensor(sequence) else sequence
        # Calculate the current sequence length
        sequence_len = tensor_sequence.size()[0]

        # Limit the sequence length within (max_size - 2) where 2 corresponding to bos and eos tags
        cutted_sequence_size = max(0, min(max_size - 2, sequence_len + 2))
        tensor_sequence = tensor_sequence[:cutted_sequence_size]
        
        # Calculate the padding size
        padding_size = max_size - tensor_sequence.size()[0] - 2 # expected size - current size - (BOS tag + EOS tag)
        
        # Add BOS, PAD, and EOS tokens
        tensor_sequence = F.pad(tensor_sequence, (1,0), "constant", self.BOS_IDX)
        tensor_sequence = F.pad(tensor_sequence, (0,padding_size), "constant", self.PAD_IDX)
        tensor_sequence = F.pad(tensor_sequence, (0,1), "constant", self.EOS_IDX)
        
        return tensor_sequence
    
    def sequence_clearner(self, sequence):
        """ Method used to remove BOS/PAD/EOS special tokens """
        # Checking tensor format
        list_sequence = sequence.tolist() if torch.is_tensor(sequence) else sequence
        def check_special(number):
            return number not in [self.BOS_IDX, self.EOS_IDX, self.PAD_IDX]
        return list(filter(CheckSpecial, list_sequence))

In [4]:
tokenizer = Tokenizer()

In [5]:
@dataclass
class TransformerConfig:
    """Data class that stores the configuration for a Transformer model.

    Attributes:
        - tokenizer: An instance of the Tokenizer class.
        - block_size (int): Number of tokens in each sequence. Defaults to 512.
        - batch_size (int): Number of sequences in each batch. Defaults to 12.
        - vocab_size (int): Total size of the tokenizer vocabulary. It is set to the size of the tokenizer vocabulary.
        - n_layer (int): Number of transformer encoder and decoder blocks (N). Defaults to 1.
        - n_head (int): Number of heads in each attention block. Defaults to 2.
        - n_embd (int): Token embedding size. This is from the original Transformer paper. Defaults to 128.
        - dropout (float): Dropout rate to use in the Transformer model. Defaults to 0.1.
        - bias (bool): Indicates whether to use bias in Linears and LayerNorms.
            If True, bias is used similar to GPT-2.
            If False, it is a bit better and faster. Defaults to False.
        - device (str): The device to run the model on. Defaults to 'cpu'. 'cuda' is used if a GPU is available.
        - learning_rate (float): Learning rate for the model optimization. Defaults to 3e-4.
        - eval_interval (int): Number of steps between each validation dataset. Defaults to 1.
        - eval_iters (int): Number of validation epochs. Defaults to 20.
    """
    tokenizer: any
    block_size: int = 512
    batch_size: int = 12
    n_layer: int = 1 # 6
    n_head: int = 2 # 8
    n_embd: int = 128 # 512
    dropout: float = 0.1
    bias: bool = False # True:
    # device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = 'cpu'
    learning_rate = 3e-4
    eval_interval = 1
    eval_iters = 20 # 200

    @property
    def vocab_size(self) -> int:
        """Returns the total size of the tokenizer vocabulary.

        Returns:
            int: The size of the tokenizer vocabulary.
        """
        return self.tokenizer.vocab_size()

In [6]:
from datasets import load_dataset, Dataset
from torch.utils.data import Dataset, DataLoader

In [8]:
class TranslationDataset(Dataset):
    """
    A PyTorch Dataset class to be used in a PyTorch DataLoader to create batches.

    Args:
        - dataset (Dataset): a dataset from HuggingFace datasets library.
        - tokenizer (Tokenizer): The custom tiktoken tokenizer used to encode sequences.
        - block_size (int): The maximum sequence length for tokenization.
    """

    def __init__(self, dataset, tokenizer, block_size):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.block_size = block_size

    def __getitem__(self, index):
        """
        Get a tokenized example from the dataset at the specified index.

        Args:
            - index (int): the index of the example to fetch.

        Returns:
            - Dict: dictionary with keys 'inputs', 'targets' and 'translation', containing tokenized input,
            target sequences and original translation.
        """
        translation = self.dataset[index]['translation']
        encode = self.tokenizer.encoder.encode
        inputs = self.tokenizer.sequence_padding(encode(translation['en']), self.block_size) # source language
        targets = self.tokenizer.sequence_padding(encode(translation['fr']), self.block_size) # target language
        return {'inputs': inputs, 'targets': targets, 'translation': translation}

    def __len__(self) -> int :
        """
        Returns the length of the dataset.

        Returns:
            - int: the length of the dataset.
        """
        return self.dataset.num_rows


class DataLoaderFactory():
    """
    A class to instantiate PyTorch DataLoaders for different splits of a HuggingFace Dataset.

    Args:
        - block_size (int): The maximum sequence length for tokenization.
        - batch_size (int): The batch size for DataLoader.
        - tokenizer (Tokenizer): a tokenizer that has an encode method.
        - device (str): 'cpu' or 'cuda', depending on whether we use CPU or GPU.
    """

    def __init__(self, block_size, batch_size, tokenizer, device):
        self.train_data = TranslationDataset(load_dataset("wmt14", "fr-en", split="train[:500000]"), tokenizer, block_size)
        self.val_data = TranslationDataset(load_dataset("wmt14", "fr-en", split="validation"), tokenizer, block_size)
        self.test_data = TranslationDataset(load_dataset("wmt14", "fr-en", split="test"), tokenizer, block_size)

        self.block_size = block_size
        self.batch_size = batch_size
        self.device = device

        self.dataloader_train = DataLoader(self.train_data, batch_size=batch_size, shuffle=True)
        self.dataloader_val = DataLoader(self.val_data, batch_size=batch_size, shuffle=True)
        self.dataloader_test = DataLoader(self.test_data, batch_size=batch_size, shuffle=True)

    
    
    def __len__(self) -> int :
        """
        Print the length of each dataset and returns the length of all datasets.

        Returns:
            - int: the length of all dataset (train + val + test).
        """
        print("\033[95m\033[1m\033[4mNumber of data by datasets splits\033[0m")
        print(f"Train\t\t: {len(self.train_data)}")
        print(f"Validation\t: {len(self.val_data)}")
        print(f"Test\t\t: {len(self.test_data)}")
        total = len(self.train_data) + len(self.val_data) + len(self.test_data)
        print(f"Total\t\t: {total}")
        return total

    def get_batch(self, split):
        """
        Choose the correct DataLoader and yield batches from it.

        Args:
            - split (str): 'train', 'val' or 'test'.

        Yields:
            - Dict: a dictionary with keys 'inputs', 'targets' and 'translation', containing a batch of tokenized input,
            target sequences and original translation.
        """
        # choose the correct dataloader
        if split == 'train':
            dataloader = self.dataloader_train
        elif split == 'val':
            dataloader = self.dataloader_val
        else:
            dataloader = self.dataloader_test

        for batch in dataloader:
            # Separate the 'translation' from the rest of the batch
            translation = batch.pop('translation')
    
            # Move tensors to device
            batch_on_device = {k: v.to(self.device) for k, v in batch.items()}
    
            # Add 'translation' back into the batch
            batch_on_device['translation'] = translation
    
            yield batch_on_device

In [9]:
config = TransformerConfig(tokenizer)
dataset = DataLoaderFactory(config.block_size, config.batch_size, tokenizer, config.device)
# dataset.DataSize()


Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)


In [10]:
batch = dataset.get_batch('train')
batch

<generator object DataLoaderFactory.get_batch at 0x0000016367F12240>

In [11]:
next(batch)

{'inputs': tensor([[100264,  10445,   1587,  ..., 100266, 100266, 100265],
         [100264,   1687,   1101,  ..., 100266, 100266, 100265],
         [100264,   1687,    527,  ..., 100266, 100266, 100265],
         ...,
         [100264,    791,   9647,  ..., 100266, 100266, 100265],
         [100264,   2173,   1521,  ..., 100266, 100266, 100265],
         [100264,  86767,     11,  ..., 100266, 100266, 100265]]),
 'targets': tensor([[100264,  43278,  62998,  ..., 100266, 100266, 100265],
         [100264,  32960,  17317,  ..., 100266, 100266, 100265],
         [100264,  66932,    841,  ..., 100266, 100266, 100265],
         ...,
         [100264,     43,      6,  ..., 100266, 100266, 100265],
         [100264,  41211,     72,  ..., 100266, 100266, 100265],
         [100264,  22560,  46138,  ..., 100266, 100266, 100265]]),
 'translation': {'en': ['Why does the cosmetics industry not announce that it will invest money - a great deal of money - in research, in order to have the means to ma

In [12]:
len(dataset)

Number of data by datasets splits
Train		: 500000
Validation	: 3000
Test		: 3003
Total		: 506003


506003

In [ ]:
class LayerNorm(nn.Module):
    """A Layer Normalization module with optional bias.

    This implementation of Layer Normalization allows turning off the bias term,
    which is not directly supported by PyTorch's layer normalization function.

    Attributes:
        weight: Learnable weights for the layer normalization. Initialized as an all ones tensor.
        bias: Learnable biases for the layer normalization. Initialized as an all zeros tensor 
              if bias argument in constructor is True, otherwise it's set to None.

    Args:
        ndim: An integer for the dimension of the input vectors.
        bias: A boolean which, if True, adds a learnable bias to the output.

    """
    def __init__(self, ndim: int, bias: bool):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        """Defines the computation performed at every call.

        Args:
            input (tensor): The input tensor to be normalized.

        Returns:
            tensor: The normalized input tensor.

        """
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

In [ ]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention module.
    
    This module applies multi-head attention mechanism on the input sequence. This implementation doesn't apply mask over the attention scores.
    
    Attributes:
        - n_head (int): Number of attention heads.
        - n_embd (int): Embedding dimensionality.
        - dropout (float): Dropout rate.
        - q_attn (nn.Linear): Linear layer for the query projection.
        - k_attn (nn.Linear): Linear layer for the key projection.
        - v_attn (nn.Linear): Linear layer for the value projection.
        - c_proj (nn.Linear): Linear layer for the output projection.
        - attn_dropout (nn.Dropout): Dropout layer for the attention scores.
        - resid_dropout (nn.Dropout): Dropout layer for the residual connection.
        - flash (bool): Flag indicating if flash attention is available.
    """
    def __init__(self, config):
        """
        Constructor for the MultiHeadAttention class.
        
        Args:
            - config: The configuration object containing model parameters.
        """
        super().__init__()
        assert config.n_embd % config.n_head == 0

        # Params
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        
        # INPUTS: query, key, value projections for all heads, but in a batch
        self.q_attn = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.k_attn = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.v_attn = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        
        # OUTPUT: output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        
        # flash attention make GPU go br but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer(
                "bias",
                torch.tril(
                    torch.ones(config.block_size, config.block_size)
                ).view(1, 1, config.block_size, config.block_size)
            )

    def scaled_dot_product_attention(self, q, k, v, mask: bool = None):
        """
        Computes the scaled dot product attention.
        
        Args:
            - q (Tensor): Query tensor of shape (batch_size, num_heads, seq_length, emb_dim).
            - k (Tensor): Key tensor of shape (batch_size, num_heads, seq_length, emb_dim).
            - v (Tensor): Value tensor of shape (batch_size, num_heads, seq_length, emb_dim).
            - mask (bool, optional): Flag indicating whether to apply mask on the attention scores.

        Returns:
            - y (Tensor): Output tensor after applying attention.
        """
        # manual implementation of attention
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1))) # Step 1 & 2: (MatMul) and (Scale)
        if mask:
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1) # Step 3: Softmax
        att = self.attn_dropout(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs) # Step 4: MatMul
        return y

    def forward(self, q_x, k_x, v_x, mask = None):
        """
        Forward pass for the MultiHeadAttention module.
        
        Args:
            - q_x (Tensor): Input query tensor of shape (batch_size, seq_length, emb_dim).
            - k_x (Tensor): Input key tensor of shape (batch_size, seq_length, emb_dim).
            - v_x (Tensor): Input value tensor of shape (batch_size, seq_length, emb_dim).
            - mask (bool, optional): Flag indicating whether to apply mask on the attention scores.

        Returns:
            - y (Tensor): Output tensor after applying multi-head attention.
        """
        B, T, C = q_x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v = self.q_attn(q_x), self.k_attn(k_x), self.v_attn(v_x)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=mask == True)
        else:
            # manual implementation of attention
            y = self.scaled_dot_product_attention(q, k, v)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side # Step 5: Concatenate

        # output projection
        y = self.resid_dropout(self.c_proj(y)) # Step 6 : Linear
        return y

In [13]:
assert 'a' = 'b'

SyntaxError: invalid syntax (2352792203.py, line 1)

In [ ]:
class FeedForward(nn.Module):
    """ Multi Layer Perceptron (FeedForward) """
    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

In [ ]:
class EBlock(nn.Module):
    """ EBlock = (Encoder block) """
    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = MultiHeadAttention(config, False)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.ffw = FeedForward(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.ffw(self.ln_2(x))
        return x
    

In [ ]:
class DBlock(nn.Module):
    # DBlock (Decoder block)
    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn1 = MultiHeadAttention(config, True, False)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn2 = MultiHeadAttention(config, False, True)
        self.ln_3 = LayerNorm(config.n_embd, bias=config.bias)
        self.ffw = FeedForward(config)

    def forward(self, x, encoder_output):
        x = x + self.attn1(self.ln_1(x))
        x = x + self.attn2(self.ln_2(x), encoder_output)
        x = x + self.ffw(self.ln_3(x))
        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([EBlock(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("Number of Encoder parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb) # Addition of input embd + positional encoding
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = x
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            logits = x
            loss = None

        return logits, loss
    

In [ ]:
class Decoder(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([DBlock(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, enc_output = None, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb) # Addition of input embd + positional encoding
        for block in self.transformer.h:
            x = block(x, enc_output)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.contiguous().view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
        # first estimate the number of flops we do per iteration.
        # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
        N = self.get_num_params()
        cfg = self.config
        L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # express our flops throughput as ratio of A100 bfloat16 peak flops
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [ ]:
class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)

    # def generate_mask(self, src, tgt):
    #     src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
    #     tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
    #     seq_length = tgt.size(1)
    #     nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
    #     tgt_mask = tgt_mask & nopeak_mask
    #     return src_mask, tgt_mask


    def forward(self, src, tgt):
        # src_mask, tgt_mask = self.generate_mask(src, tgt)
        enc_output, _ = self.encoder(src)
        output, loss = self.decoder(tgt, enc_output, tgt)
        return output, loss

--------

In [ ]:
@torch.no_grad()
def estimate_loss(dataset, config):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(config.eval_iters)
        for k in range(config.eval_iters):
            X, Y, _ = dataset.GetBatch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
config = TransformerConfig(tokenizer)
model = Transformer(config)
dataset = Data(config.block_size, config.batch_size, tokenizer, config.device)
dataset.DataSize()

In [ ]:
# create a PyTorch optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, betas=(0.9, 0.98), eps=1e-9)

model.train()
model = model.to(config.device)

In [ ]:
max_iters = 5

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % config.eval_interval == 0:
        losses = estimate_loss(dataset, config)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb, _ = dataset.GetBatch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
model.generate()